## Scrape CCNY 2021 Fall Calendar

* Use:
    * requests library
    * BeautifulSoup
    * Pandas

* End up with Pandas data frame:
    * index column is python date
    * "D.O.F" Column
    * Text column with explaination.

In [ ]:
!pip install pandas
!pip install beautifulsoup4
!pip install requests

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


# Added the imports to start the project, will add more as needed.

1. Lets set the URL and make the make the request.

In [ ]:
url = 'https://www.ccny.cuny.edu/registrar/fall'

response = requests.get(url)
html_document = response.text

print(html_document)